In [48]:
import pandas as pd
from datasets import load_dataset, Dataset

data_frame = pd.read_json('datasets/tobyoki/tobyoki-event_summary_juman_processed_grouped.json', orient='records', encoding='utf-8', lines=False)
# def truncate_max_segments(examples):
#     return examples[:10]
# for column in data_frame.columns:
#     data_frame[column] = data_frame[column].apply(truncate_max_segments)
raw_datasets = Dataset.from_pandas(data_frame)
raw_datasets = raw_datasets.train_test_split(test_size=0.1, seed=42)
temp = raw_datasets['train'].train_test_split(test_size=0.1/(0.8+0.1), seed=42)
raw_datasets['train'], raw_datasets['validation'] = temp['train'], temp['test']

In [49]:
# 删除'user'和'summary'列
for split in ['train', 'test', 'validation']:
    raw_datasets[split] = raw_datasets[split].remove_columns(['user', 'summary'])



In [50]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'incremental_summary'],
        num_rows: 80
    })
    test: Dataset({
        features: ['text', 'incremental_summary'],
        num_rows: 10
    })
    validation: Dataset({
        features: ['text', 'incremental_summary'],
        num_rows: 10
    })
})

In [51]:
raw_datasets['train'][0]

{'text': ['2021 - 05 - 01 \n さて と 。 \n 帰る 電車 の 中 で は 、 涙 たれ流し 状態 の 頭 で 家族 に どう 伝える の が いい かなー と グルグル 。 とりあえず 、 子ども を 迎え に 行って 。 ご飯 準備 して 。 やる 事 が ある って いう 有り難 さ が 身 に 染みる 。 コドモ に は 検査 の 時 から 、 おっぱい の 中 に ワルモノ が い ない か 調べて る よー って 話して た から 、 「 戦う ワルモノ が いた よ 。 これ から やっつける 為 に かーちゃん 頑張る し 、 色々 と 協力 して ね 。 」 と 説明 して 、 分かった ーと 。 オット も 帰って きて 「 やっぱり 良く ない 結果 だった 」 と 伝える と 、 オット は そう かー と 。 こう なったら やる しか ない ！ と 、 みんな で 気分 を 奮い立た せて ました 。 その 日 、 オット は めちゃくちゃ 早く 就寝 。 次の 日 の 朝 、 検査 ・ 治療 など に ついて 話す 。 色々 考え そうで 、 考え たく なくて 早く 寝て しまった わーと 。 そりゃ 私 も 不安 満開 だった から 、 オット も 不安 よ なぁ 。 告知 さ れて から 、 なんだか 一気に 病気 の 自分 が 押し寄せて きて 、 色んな 気持ち が グールグル 。 病気 に なった 意味 と か 考えて しまう なー 。 ちょっと 前 に 10 年 生存 率 が 発表 さ れて た なー 。 その 時 は ニュース 流し 見 して た なー 。 考えて も 仕方ない 事 ばかり 考えて しまう 。 頭 の 中 が グールグル 。 10 年 後 。 どう なる ？ ワタシ 。',
  '2021 - 05 - 01 \n 乳 がん 告知 \n 診察 室 へ 入る 。 造影剤 検査 ビックリ し ました ねー と か 話し ながら 、 本題 の 検査 結果 へ 。 先生 「 この前 の 生検 の 結果 、 右 外 側 の 腫瘤 から ガン が みつかり ました 。 浸潤 がん 、 大き さ は 1 . 8 cm 位 。 ステージ 1 です 。 」 と 。 そう かー 、 が んかー 

In [52]:
from datasets import Dataset

new_datasets = {}

for split in ['train', 'test', 'validation']:
    # 初始化新的列表，用于存储拆分后的数据
    new_texts = []
    new_incremental_summaries = []

    # 遍历每一行数据
    for data in raw_datasets[split]:
        # 拆分'text'列中的列表
        text_sentences = data['text']
        # 拆分'incremental_summary'列中的列表
        incremental_summary_sentences = data['incremental_summary']
        
        # 将拆分后的句子添加到新的列表中
        new_texts.extend(text_sentences)
        new_incremental_summaries.extend(incremental_summary_sentences)
    
    # 创建新的数据集对象
    new_dataset = Dataset.from_dict({
        'text': new_texts,
        'incremental_summary': new_incremental_summaries
    })
    
    # 覆盖原始数据集
    raw_datasets[split] = new_dataset


In [55]:
raw_datasets['test'][6]

{'text': '2023 - 03 - 13 \n 今日 の 夢 \n 大学 病院 受診 まで の 待機 期間 。 毎日 仕事 行って 、 家 の こと して 、 子供 の 参観 日 行って 、 ヨーカドー 行って 、  、 普通の 日々 。 あれ ？ 私 病気 な の ？ って 感じ このまま 普通に 生きて いけたら なー と 何度 も 現実 逃避 して ます 。 お 年寄り を 見る と 、 よくぞ その 年齢 まで 生きて くれ ました パチパチ 同じ 世代 の 人 たち を 見る と 、 健康な んだろう なー 、 いい なー 。 検診 いこう ねー 今日 、 電車 の 中 で ウィッグ が 取れて しまい 、 焦って 『 すみません すみません 』 と 周り に 謝る 夢 を 見 ました 。 ふわふわ し ながら も 、 頭 の 中 、 がん で いっぱいです でも 、 ここ で 皆さん の ブログ を 読んで 、 治療 を し ながら 働いて いたり 、 子供 の 行事 に 参加 して いたり 、 前向きに 治療 と 日常 を 両立 さ れて いる ！ 今 より でき なく なる かも しれ ない けど 、 全て を 諦める 必要 は ない んだ と いう こと が わかり ました 。 私 は 本当 は 春 から 転職 希望 で 。 短 時間 パート から フルタイム 勤務 を 探して いた ところ でした 。 タイミング は 今 で は なく なって しまった けど 、 諦め ませ ん ここ に 宣言 して み ます 。 上 の 子 の 反抗 期 も これ から 始まる し 、 高校 生 に なったら お 弁当 も 作ら ない と いけない 。 下 の 子 が 1 人 で 眠れる まで 付き合わ ない と いけない し 、 女の子 だ から あと 数 年 後 身体 の 変化 と か 下着 選び と か 一緒に して あげ たい 。 どんな 形 でも 生き なきゃ 頑張り ます 週 末 は 現実 逃避 の ため 、 B リーグ 観て き ます WBC も 面白い ！ スポーツ 観戦 して いる と 、 一瞬 病気 の こと 忘れ られ ます 心 に 栄養 を',
 'incremental_summary': '2023 - 02 - 08 乳 が